In [1]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

from qiskit.result import QuasiDistribution
from qiskit_aer.primitives import Sampler
from qiskit_algorithms import NumPyMinimumEigensolver, QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms.utils import algorithm_globals

In [2]:
def get_portfolio_data(tickers, start_date, end_date):
    """
    Fetches historical data and calculates expected return (mu) and covariance (sigma).
    """
    #Fetch raw data from Yahoo Finance

    data = yf.download(tickers, start=start_date, end=end_date)['Close']
    
    #Calculate daily returns (percentage change)
    daily_returns = data.pct_change().dropna()
    
    #Calculate Mean Vector (mu) and Covariance Matrix (sigma)
    '''252 because if not, mu is the average daily returns and overall we want annual returns because it makes more sense.
    So just multiply the average daily returns by the 252 annual trading days'''
    mu = daily_returns.mean() * 252
    sigma = daily_returns.cov() * 252
    
    return mu, sigma, data

def normalize_data(mu, sigma):
    """
    Normalizes data to [0, 1] range.
    """
    mu_min = mu.min()
    mu_max = mu.max()
    mu_normalized = (mu - mu_min) / (mu_max - mu_min) #Normalization between [0,1]

    sigma_max = sigma.max().max()
    sigma_normalized = sigma / sigma_max #Normalization of sigma (the covariance matrix)
    
    return mu_normalized, sigma_normalized

def create_portfolio_qp(mu, sigma, q=0.5, budget=None):
    """
    Creates a Quadratic Program for the Portfolio Optimization problem.
    
    Args:
        mu (numpy.ndarray): Expected returns vector.
        sigma (numpy.ndarray): Covariance matrix.
        q (float): Risk factor (0 = high risk/high return, 1 = low risk).
        budget (int): Number of assets to select. If None, defaults to half the assets.
        
    Returns:
        qp (QuadraticProgram): The mathematical formulation of the problem.
        penalty (float): The recommended penalty scaling factor for QUBO conversion.
    """
    num_assets = len(mu)
    
    if budget is None:
        budget = num_assets // 2
        
    # Set parameter to scale the budget penalty term
    penalty = num_assets 
    
    # Create the portfolio instance
    portfolio = PortfolioOptimization(
        expected_returns=mu.values, 
        covariances=sigma.values, 
        risk_factor=q, 
        budget=budget
    )
    
    # Convert to Qiskit's QuadraticProgram format
    qp = portfolio.to_quadratic_program()
    
    return qp, penalty,portfolio

def print_result(result,portfolio):
    selection = result.x
    value = result.fval
    print("Optimal: selection {}, value {:.4f}".format(selection, value))

    eigenstate = result.min_eigen_solver_result.eigenstate
    probabilities = (
        eigenstate.binary_probabilities()
        if isinstance(eigenstate, QuasiDistribution)
        else {k: np.abs(v) ** 2 for k, v in eigenstate.to_dict().items()}
    )
    print("\n----------------- Full result ---------------------")
    print("selection\tvalue\t\tprobability")
    print("---------------------------------------------------")
    probabilities = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)

    for k, v in probabilities:
        x = np.array([int(i) for i in list(reversed(k))])
        value = portfolio.to_quadratic_program().objective.evaluate(x)
        print("%10s\t%.4f\t\t%.4f" % (x, value, v))

In [4]:
#Parameters
tickers = ["AAPL", "GOOG", "MSFT","AMZN","META"]#["MRNA", "PFE","PYPL","ENPH", "SEDG"]+["AAPL", "GOOG", "MSFT", "TSLA","AMZN","META","NVDA","NFLX"]#+["MRNA", "PFE", "ENPH", "SEDG", "WBA", "PYPL"]
start_date = "2023-01-01"
end_date = "2024-01-01"
n_stocks = len(tickers)//2
risk = 0.2


mu_raw, sigma_raw, prices = get_portfolio_data(tickers, start_date, end_date)

#Normalization
#print("Expected Returns (mu):\n", mu_raw)
mu_norm, sigma_norm = normalize_data(mu_raw, sigma_raw)


qp_realdata, penalty,portfolio_qp = create_portfolio_qp(mu_norm,sigma_norm,q=risk,budget=n_stocks)

# Exact Solver Implementation
exact_mes = NumPyMinimumEigensolver()
exact_eigensolver = MinimumEigenOptimizer(exact_mes)

result = exact_eigensolver.solve(qp_realdata)

print_result(result,portfolio_qp)


# QAOA Implementation
algorithm_globals.random_seed = 1234

cobyla = COBYLA()
cobyla.set_options(maxiter=250)
qaoa_mes = QAOA(sampler=Sampler(), optimizer=cobyla, reps=3)
qaoa = MinimumEigenOptimizer(qaoa_mes, penalty=20)
result = qaoa.solve(qp_realdata)


print_result(result,portfolio_qp)

C:\Users\andre\AppData\Local\Temp\ipykernel_6428\28816047.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date, end=end_date)['Close']
[*********************100%***********************]  5 of 5 completed


Optimal: selection [0. 1. 0. 1. 0.], value -0.7247

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
[0 1 0 1 0]	-0.7247		1.0000
Optimal: selection [0. 1. 0. 1. 0.], value -0.7247

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
[1 0 0 1 0]	-0.6415		0.0938
[0 1 0 1 0]	-0.7247		0.0889
[0 0 0 1 1]	-0.6376		0.0654
[1 0 0 1 1]	-0.4098		0.0654
[0 1 0 1 1]	-0.4418		0.0635
[0 0 1 1 0]	-0.5570		0.0615
[1 1 0 1 0]	-0.4928		0.0566
[0 1 1 1 0]	-0.3256		0.0557
[1 0 1 1 0]	-0.3159		0.0537
[0 1 0 0 1]	0.0316		0.0430
[0 0 1 1 1]	-0.2941		0.0420
[1 1 0 0 0]	0.0066		0.0381
[1 0 0 0 1]	0.1476		0.0371
[1 0 1 0 0]	0.1882		0.0361
[0 1 1 0 0]	0.0944		0.0342
[0 0 1 0 1]	0.1840		0.0283
[0 1 1 1 1]	0.0577		0.0234
[1 1 0 1 1]	-0.1407		0.0146
[0 1 0 0 0]	-0.1170		0.0127
[1 1 0 0 1]	0.2245		0.0117
[1 0 0 0 0]	0.0502		0.0107
[1 0 1 0 1]	0.3860		0.0